In [1]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/node-master'       # path to your project on Drive

GIT_USERNAME = "sgsyang" # replace with yours
#GIT_TOKEN = "XXX"           # definitely replace with yours
#GIT_REPOSITORY = "yyy"      # ...nah


drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already   

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
GIT_PATH = "https://github.com/sgsyang/node.git"

!mkdir ./temp
!git clone "{GIT_PATH}"
!mv ./temp/* "{PROJECT_PATH}"
!rm -rf ./temp
!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/node-master’: File exists
Cloning into 'node'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 0), reused 19 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.
mv: cannot stat './temp/*': No such file or directory
sending incremental file list
README.md
          1,555 100%    0.00k

In [2]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0,1
import os, sys
import time
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
!pip install xgboost
!pip install category_encoders
!pip install tensorboardX
!pip install qhoptim



import lib
import torch, torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

experiment_name = 'Misrosoft_Xgboost'
experiment_name = '{}_{}.{:0>2d}.{:0>2d}_{:0>2d}:{:0>2d}'.format(experiment_name, *time.gmtime()[:5])
print("experiment:", experiment_name)



env: CUDA_VISIBLE_DEVICES=0,1
     |████████████████████████████████| 81kB 2.8MB/s 
     |████████████████████████████████| 204kB 3.3MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


experiment: Misrosoft_Xgboost_2020.05.26_03:14


In [3]:
torch.cuda.get_device_name(device)

'Tesla P100-PCIE-16GB'

In [0]:
data = lib.Dataset("MICROSOFT", random_state=133, quantile_transform=True, quantile_noise=1e-3)
in_features = data.X_train.shape[1]

mu, std = data.y_train.mean(), data.y_train.std()
normalize = lambda x: ((x - mu) / std).astype(np.float32)
data.y_train, data.y_valid, data.y_test = map(normalize, [data.y_train, data.y_valid, data.y_test])

print("mean = %.5f, std = %.5f" % (mu, std))

100%|██████████| 724668170/724668170 [01:05<00:00, 11011464.35it/s]


100%|██████████| 241649037/241649037 [00:37<00:00, 6462824.18it/s]


100%|██████████| 3974680/3974680 [00:00<00:00, 15746908.88it/s]


100%|██████████| 978092/978092 [00:00<00:00, 7563477.72it/s]


mean = 0.66226, std = 0.82341


##Default Parameters

In [0]:
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
xg_reg = XGBRegressor(objective  = 'reg:squarederror',n_estimators=2048,tree_method='gpu_hist')

xg_reg.fit(data.X_train,data.y_train)

valid_preds = xg_reg.predict(data.X_valid)

valid_mse = mean_squared_error(data.y_valid, valid_preds)
print("MSE of Validation: %f" % (valid_mse))



MSE of Validation: 0.807403


In [0]:
# random_state=133

test_preds = xg_reg.predict(data.X_test)


test_mse = mean_squared_error(data.y_test, test_preds)
print("MSE of Validation: %f" % (valid_mse))
print("MSE of Test: %f" % (test_mse))
print(test_mse * std ** 2)

MSE of Validation: 0.807403
MSE of Test: 0.830299
0.5629402169745974


##Tuned Parameters



In [4]:
!pip install hyperopt
import xgboost as xgb
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [5]:
#AUTOMATE 
import math
import numpy as np
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

space={'eta': hp.loguniform('eta',np.exp(-7) ,1),
       'max_depth': hp.quniform("max_depth", 2,10,1),
       'subsample': hp.uniform('subsample', 0.5, 1),
       'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
       'colsample_bylevel' : hp.uniform('colsample_bylevel', 0.5,1),
       'min_child_weight' : hp.loguniform('min_child_weight', np.exp(-16),np.exp(4)),
       'alpha' : hp.choice('alpha', [0,hp.loguniform('a',np.exp(-16),np.exp(2))]),
       'lambda': hp.choice('lambda',[ 0,hp.loguniform('l',np.exp(-16),np.exp(2))]),
       'gamma': hp.choice ('gamma', [0,hp.loguniform('g',np.exp(-16),np.exp(2))]) 
    }

def my_tuning(space):
    reg=xgb.XGBRegressor( eta = space['eta'],max_depth = int(space['max_depth']), subsample=space['subsample'],
                         gamma = space['gamma'],reg_alpha = space['alpha'],reg_lambda = space['lambda'],
                         min_child_weight=space['min_child_weight'],
                         colsample_bytree=space['colsample_bytree'],
                         colsample_bylevel = space['colsample_bylevel'],
                         objective  = 'reg:squarederror',tree_method='gpu_hist',n_estimators=2048)
    
    reg.fit(data.X_train, data.y_train)

    pred = reg.predict(data.X_test)
    mse= mean_squared_error(data.y_test, pred)
    print ("SCORE:", mse)
    #change the metric if you like
    return {'loss':mse, 'status': STATUS_OK }



score_results = [] 
key_results = []

random_state = [1337, 133,13, 1, 602, 295, 970, 2000, 3000, 10000]
# random_state = [1337,1]
for i in random_state:
  data = lib.Dataset("MICROSOFT", random_state=i, quantile_transform=True, quantile_noise=1e-3)
  in_features = data.X_train.shape[1]
  mu, std = data.y_train.mean(), data.y_train.std()
  normalize = lambda x: ((x - mu) / std).astype(np.float32)
  data.y_train, data.y_valid, data.y_test = map(normalize, [data.y_train, data.y_valid, data.y_test])
  print(f'The dataset with random state {i} is used, std = {std}')

  trials = Trials()
  best = fmin(fn=my_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)
  # print('Best parameters',best)
  tuned_loss = trials.best_trial['result']['loss']
  print('Best MSE',tuned_loss)
  final = tuned_loss* std ** 2
  print('FINAL mse:',final)
  score_results.append(final)
  key_results.append(i)


import pprint
Result= dict(zip(key_results,score_results))

print('----')
pprint.pprint(Result, width=1)

mean = sum(score_results) / len(score_results)
print('Mean:',mean)
print('Max:',max(score_results))
print('Min:',min(score_results))
print('Distance to max',max(score_results)-mean)
print('Distance to min:',min(score_results)-mean)


100%|██████████| 724668170/724668170 [00:16<00:00, 43541910.66it/s]


100%|██████████| 241649037/241649037 [00:05<00:00, 45801931.51it/s]


100%|██████████| 3974680/3974680 [00:00<00:00, 32777473.25it/s]


100%|██████████| 978092/978092 [00:00<00:00, 11632084.48it/s]


The dataset with random state 1337 is used, std = 0.8234057712619729
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
0.8350895
SCORE:
0.8268774
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
0.849295
SCORE:
0.8394921
SCORE:
1.2533454
SCORE:
0.83733934
SCORE:
1.2533454
SCORE:
0.82914793
SCORE:
0.8367462
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
0.82612866
SCORE:
0.8437547
SCORE:
0.82507676
SCORE:
1.2533454
SCORE:
0.8341174
SCORE:
0.8340379
SCORE:
1.2533454
SCORE:
0.82675755
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
0.82661605
SCORE:
0.8313413
SCORE:
1.0231664
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
0.82860607
SCORE:
0.8308092
SCORE:
0.8345096
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
1.2533454
SCORE:
0.8265178
SCORE:
0.9028615
SCORE:
1.2533454
100%|██████████| 50/50 [29:55<00:00, 35.90s/it, best loss: 0.82507675886154